# Customer Behavior Metric Views - Sample Queries

This notebook contains sample queries demonstrating how to use each customer behavior metric view with the `MEASURE()` function.

**Catalog/Schema:** Set in the **Configuration** cell below (session variables `catalog_name`, `schema_name`). Run that cell first so queries resolve to the correct metric views.  
**Purpose:** Query examples for Customer Behavior Genie Room metric views  
**Created:** 2025-01-29

## Overview

These queries demonstrate best practices for querying metric views:
- Use `MEASURE()` function to access pre-defined measures
- Dimensions can be selected directly without aggregation
- GROUP BY dimensions to aggregate measures
- Use WHERE clause to filter dimensions

**Note:** Change the `DEFAULT` values in the Configuration cell to use a different catalog/schema.


In [0]:
-- =============================================================================
-- Configuration: Set target catalog and schema (same as metric views notebook)
-- Change these defaults to query metric views in a different catalog/schema
-- =============================================================================
DECLARE OR REPLACE VARIABLE catalog_name STRING DEFAULT 'juan_use1_catalog';
DECLARE OR REPLACE VARIABLE schema_name STRING DEFAULT 'retail';

-- Set session context so unqualified view names resolve to the target location
USE CATALOG IDENTIFIER(catalog_name);
USE SCHEMA IDENTIFIER(schema_name);

-- Verify the target
SELECT current_catalog(), current_schema(),
       catalog_name || '.' || schema_name AS target;

current_catalog(),current_schema(),target
juan_use1_catalog,retail,juan_use1_catalog.retail


## 1. Customer Segmentation Queries

**View:** `customer_segmentation_mv`


In [0]:
-- What are the key customer segments?
SELECT 
  Segment,
  MEASURE(`Customer Count`) as customer_count,
  MEASURE(`Average Lifetime Value`) as avg_cltv,
  MEASURE(`Total Lifetime Value`) as total_cltv
FROM customer_segmentation_mv
GROUP BY Segment
ORDER BY MEASURE(`Average Lifetime Value`) DESC;


Segment,customer_count,avg_cltv,total_cltv
vip,2500,14775.647199999989,3.693911799999997E7
premium,7500,4993.883006666656,3.745412254999992E7
loyal,12500,1900.0410431999965,2.3750513039999954E7
regular,17500,699.9626674285756,1.2249346680000074E7
new,10000,277.7368239999997,2777368.2399999965


In [0]:
-- Show customer distribution by segment and region
SELECT 
  Segment,
  `Geo Region`,
  MEASURE(`Customer Count`) as customer_count,
  MEASURE(`Average Lifetime Value`) as avg_cltv
FROM customer_segmentation_mv
GROUP BY Segment, `Geo Region`
ORDER BY Segment, MEASURE(`Customer Count`) DESC;


Segment,Geo Region,customer_count,avg_cltv
loyal,Southeast,3212,1911.6976587795778
loyal,Northeast,3131,1891.3892206962623
loyal,Midwest,3111,1894.2407457409206
loyal,West,3046,1902.5664970453101
new,West,2047,276.3952564728872
new,Midwest,2007,276.83852017937267
new,Southwest,2004,281.0283133732536
new,Northeast,1974,275.8058763931104
new,Southeast,1968,278.63348577235735
premium,Northeast,2597,5018.89983057375


## 2. RFM Analysis Queries

**View:** `customer_rfm_analysis_mv`


In [0]:
-- Which customers are at risk of churning?
SELECT 
  Segment,
  `Loyalty Status`,
  MEASURE(`Customer Count`) as customer_count,
  MEASURE(`Average Recency Days`) as avg_recency,
  MEASURE(`Average Frequency`) as avg_frequency,
  MEASURE(`Average Monetary Value`) as avg_monetary
FROM customer_rfm_analysis_mv
WHERE `Loyalty Status` = 'At Risk'
GROUP BY Segment, `Loyalty Status`
ORDER BY MEASURE(`Customer Count`) DESC;


Segment,Loyalty Status,customer_count,avg_recency,avg_frequency,avg_monetary


In [0]:
-- Show champions by segment
SELECT 
  Segment,
  MEASURE(`Champions Count`) as champion_count,
  MEASURE(`Average Monetary Value`) as avg_value
FROM customer_rfm_analysis_mv
GROUP BY Segment
ORDER BY MEASURE(`Champions Count`) DESC;


Segment,champion_count,avg_value
loyal,256,1677.0419933237904
regular,168,986.1929287231661
premium,119,1740.7714407873545
new,53,733.4351631244101
vip,47,2933.2542202268364


## 3. Purchase Summary Queries

**View:** `customer_purchase_summary_mv`


In [0]:

-- What is average order value by segment?
SELECT 
  Segment,
  MEASURE(`Transaction Count`) as transaction_count,
  MEASURE(`Average Order Value`) as avg_order_value,
  MEASURE(`Total Revenue`) as total_revenue
FROM customer_purchase_summary_mv
GROUP BY Segment
ORDER BY MEASURE(`Average Order Value`) DESC;


Segment,transaction_count,avg_order_value,total_revenue
vip,4753,285.25051381037827,6206765.930000021
premium,13144,283.80201988584014,1.0789301389999984E7
loyal,22948,281.0662441457144,1.7583785300000038E7
regular,26484,276.2335782583696,1.3462519669999903E7
new,13398,271.45075154065825,5417885.549999998


In [0]:

-- Show purchase behavior by channel and segment
SELECT 
  `Channel Name`,
  Segment,
  MEASURE(`Transaction Count`) as transactions,
  MEASURE(`Average Order Value`) as avg_order_value,
  MEASURE(`Transactions Per Customer`) as frequency
FROM customer_purchase_summary_mv
GROUP BY `Channel Name`, Segment
ORDER BY MEASURE(`Total Revenue`) DESC;


Channel Name,Segment,transactions,avg_order_value,frequency
Physical Store - POS,loyal,9751,281.2551015632608,1.8287696924231058
Website - Desktop,loyal,8374,280.6326011646269,1.7110747854515734
iOS Mobile App,regular,10210,277.05681238121167,1.6055983645227237
Website - Desktop,regular,9984,275.95385413218696,1.5971844504879219
iOS Mobile App,premium,5115,281.80246209288543,1.7487179487179487
Website - Desktop,premium,5029,284.8028214931208,1.6978392977717758
Physical Store - POS,vip,1986,285.6668906024101,1.8986615678776291
Website - Desktop,vip,1771,286.15892817831514,1.7294921875000000
iOS Mobile App,new,3621,273.0593294405735,1.5278481012658228
Website - Desktop,new,3613,267.7755162019585,1.4942100909842845


## 4. Product Affinity Queries

**View:** `product_affinity_mv`


In [0]:

-- Show products with high affinity
SELECT 
  `Product Category Level 1`,
  `Affinity Level`,
  MEASURE(`Customer Count`) as customers,
  MEASURE(`Average Affinity Score`) as avg_affinity,
  MEASURE(`Total Predicted CLTV Impact`) as predicted_impact
FROM product_affinity_mv
WHERE `Affinity Level` = 'High Affinity'
GROUP BY `Product Category Level 1`, `Affinity Level`
ORDER BY MEASURE(`Average Affinity Score`) DESC;


Product Category Level 1,Affinity Level,customers,avg_affinity,predicted_impact


In [0]:

-- How does category affinity differ by segment?
SELECT 
  Segment,
  `Product Category Level 1`,
  MEASURE(`Average Affinity Score`) as avg_affinity,
  MEASURE(`Customer Count`) as customer_count
FROM product_affinity_mv
GROUP BY Segment, `Product Category Level 1`
ORDER BY Segment, MEASURE(`Average Affinity Score`) DESC;


Segment,Product Category Level 1,avg_affinity,customer_count
loyal,footwear,0.050092,5832
loyal,apparel,0.050054,9741
loyal,accessories,0.050036,8287
new,apparel,0.049981,5921
new,accessories,0.049938,4113
new,footwear,0.049937,2466
premium,apparel,0.050084,5783
premium,footwear,0.050052,3461
premium,accessories,0.050039,4930
regular,apparel,0.050011,11702


## 5. Channel Behavior Queries

**View:** `channel_behavior_mv`


In [0]:

-- Through which channels do customers prefer to shop?
SELECT 
  `Channel Name`,
  `Is Digital`,
  MEASURE(`Unique Customers`) as unique_customers,
  MEASURE(`Transaction Count`) as transactions,
  MEASURE(`Total Revenue`) as revenue,
  MEASURE(`Average Order Value`) as avg_order_value
FROM channel_behavior_mv
GROUP BY `Channel Name`, `Is Digital`
ORDER BY MEASURE(`Total Revenue`) DESC;


Channel Name,Is Digital,unique_customers,transactions,revenue,avg_order_value
Website - Desktop,Digital,17549,28771,1.8287007509999465E7,279.82322668012404
Physical Store - POS,Physical,8866,14313,1.3404370269999804E7,281.87681940530354
iOS Mobile App,Digital,12639,19952,1.1470797759999752E7,278.8234749635331
Facebook Shop,Digital,4673,5880,2974572.210000015,276.2161955613348
In-Store Kiosk,Digital,2930,3017,1883788.6999999871,279.9923751486307
Amazon Marketplace,Digital,2882,2951,1841746.2899999996,282.5631006443694
Website - Mobile,Digital,2887,2968,1813355.0900000005,276.5103827386399
Android Mobile App,Digital,2806,2875,1784620.009999996,278.28161702791147


## 6. Channel Migration Queries

**View:** `channel_migration_mv`


In [0]:

-- How do customers migrate between channels?
SELECT 
  `Acquisition Channel`,
  `Preferred Channel`,
  `Migration Status`,
  MEASURE(`Customer Count`) as customer_count,
  MEASURE(`Average CLTV`) as avg_cltv,
  MEASURE(`Migration Rate`) as migration_rate
FROM channel_migration_mv
GROUP BY `Acquisition Channel`, `Preferred Channel`, `Migration Status`
ORDER BY MEASURE(`Customer Count`) DESC;


Acquisition Channel,Preferred Channel,Migration Status,customer_count,avg_cltv,migration_rate
web,web,Same Channel,7067,1764.2243639450971,0E-14
social,web,Migrated,5406,2708.2844395116513,100.00000000000000
social,app,Migrated,5142,1689.047541812523,100.00000000000000
web,app,Migrated,4935,1713.5760364741627,100.00000000000000
search,app,Migrated,3755,602.0930199733673,100.00000000000000
search,web,Migrated,3730,603.1971608579087,100.00000000000000
store,web,Migrated,3708,4413.702003775624,100.00000000000000
store,store,Same Channel,2451,3911.804169726642,0E-14
email,web,Migrated,2183,1894.6860604672443,100.00000000000000
email,store,Migrated,2111,1895.9158313595428,100.00000000000000


## 7. Engagement Funnel Queries

**View:** `engagement_funnel_mv`


In [0]:

-- What are the conversion rates at each funnel stage?
SELECT 
  Segment,
  `Channel Name`,
  MEASURE(Views) as views,
  MEASURE(`Add to Cart`) as add_to_cart,
  MEASURE(Purchases) as purchases,
  MEASURE(`Cart Conversion Rate`) as cart_conv_rate,
  MEASURE(`Purchase Conversion Rate`) as purchase_conv_rate,
  MEASURE(`Overall Conversion Rate`) as overall_conv_rate
FROM engagement_funnel_mv
GROUP BY Segment, `Channel Name`
ORDER BY MEASURE(`Overall Conversion Rate`) DESC;


Segment,Channel Name,views,add_to_cart,purchases,cart_conv_rate,purchase_conv_rate,overall_conv_rate
loyal,Amazon Marketplace,0,12,1,null,8.33333333333333,null
new,Website - Desktop,0,20,1,null,5.00000000000000,null
new,Website - Mobile,0,16,1,null,6.25000000000000,null
regular,Amazon Marketplace,0,18,1,null,5.55555555555556,null
regular,Website - Desktop,0,13,1,null,7.69230769230769,null
premium,Android Mobile App,0,6,0,null,0E-14,null
regular,Website - Mobile,0,19,0,null,0E-14,null
premium,Facebook Shop,0,10,2,null,20.00000000000000,null
premium,Website - Desktop,0,12,2,null,16.66666666666667,null
regular,In-Store Kiosk,0,21,1,null,4.76190476190476,null


## 8. Cart Abandonment Queries

**View:** `cart_abandonment_mv`


In [0]:

-- What is the cart abandonment rate by segment?
SELECT 
  Segment,
  `Abandonment Stage`,
  MEASURE(`Total Carts`) as total_carts,
  MEASURE(`Abandonment Rate`) as abandonment_rate,
  MEASURE(`Recovery Rate`) as recovery_rate,
  MEASURE(`Lost Revenue`) as lost_revenue,
  MEASURE(`Recovered Revenue`) as recovered_revenue
FROM cart_abandonment_mv
GROUP BY Segment, `Abandonment Stage`
ORDER BY MEASURE(`Lost Revenue`) DESC;


Segment,Abandonment Stage,total_carts,abandonment_rate,recovery_rate,lost_revenue,recovered_revenue
regular,payment,358,98.04469273743017,2.47349823321555,100302.50000000004,1808.858227061319
regular,cart,346,97.97687861271676,2.44755244755245,91173.4,2015.037650347866
regular,shipping,308,98.37662337662338,2.11864406779661,85738.45000000001,1663.5839913984496
loyal,shipping,268,98.50746268656716,1.94174757281553,72614.85000000003,780.6312938048666
loyal,payment,244,97.13114754098361,3.64583333333333,64129.24000000003,1637.2300449705872
new,cart,220,98.18181818181818,2.27272727272727,60620.870000000046,571.0885747173196
loyal,cart,224,97.76785714285714,2.87356321839080,58711.21000000001,1280.6210076902423
new,shipping,217,98.15668202764977,2.27272727272727,56339.750000000015,862.6313247866108
new,payment,187,95.72192513368984,5.36912751677852,50895.91999999999,1466.2530110934072
premium,payment,160,97.50000000000000,3.12500000000000,41310.01,1163.209417700986


## 9. Personalization Impact Queries

**View:** `personalization_impact_mv`


In [0]:

-- Which segments respond best to personalization?
SELECT 
  Segment,
  `Affinity Level`,
  MEASURE(`Customers with Affinity`) as customers,
  MEASURE(`Average Affinity Score`) as avg_affinity,
  MEASURE(`Total Predicted CLTV Impact`) as predicted_impact,
  MEASURE(`Current Average CLTV`) as current_cltv
FROM personalization_impact_mv
GROUP BY Segment, `Affinity Level`
ORDER BY MEASURE(`Total Predicted CLTV Impact`) DESC;


Segment,Affinity Level,customers,avg_affinity,predicted_impact,current_cltv
loyal,Low Affinity,10505,0.050054,1.9347020399999876E7,1905.192756236892
regular,Low Affinity,13674,0.049999,1.4811010979999933E7,698.2310238641946
premium,Low Affinity,6213,0.050065,1.1872057829999976E7,4987.380440177022
vip,Low Affinity,2116,0.050113,6830792.409999983,14722.804664729747
new,Low Affinity,7403,0.049962,5960427.9099999955,276.72602618559677


## 10. Segment Trends Queries

**View:** `segment_trends_daily_mv`


In [0]:

-- Show monthly revenue trends by segment
SELECT 
  Segment,
  Year,
  `Month Name`,
  MEASURE(`Total Revenue`) as revenue,
  MEASURE(`Transaction Count`) as transactions,
  MEASURE(`Unique Customers`) as customers,
  MEASURE(`Average Order Value`) as avg_order_value
FROM segment_trends_daily_mv
GROUP BY Segment, Year, `Month Name`
ORDER BY Year DESC, `Month Name`, Segment;


Segment,Year,Month Name,revenue,transactions,customers,avg_order_value
loyal,2026,February,1289112.450000001,1647,1547,280.4247226452036
new,2026,February,388939.4700000001,922,888,274.48092448835575
premium,2026,February,804516.4800000001,901,864,285.99945965161754
regular,2026,February,935378.5700000025,1749,1657,280.6416351635171
vip,2026,February,432296.59,321,298,282.17793080939947
loyal,2026,January,2248579.4699999983,2897,2562,282.165826327017
new,2026,January,630150.5099999986,1565,1449,266.5611294416238
premium,2026,January,1314928.0900000003,1583,1417,282.5371916630856
regular,2026,January,1606217.3600000013,3080,2832,275.13144227475186
vip,2026,January,825526.1400000004,625,561,287.43946378830094


In [0]:

-- Compare peak vs regular season performance
SELECT 
  Segment,
  `Peak Season Indicator`,
  Season,
  MEASURE(`Total Revenue`) as revenue,
  MEASURE(`Average Order Value`) as avg_order_value,
  MEASURE(`Unique Customers`) as customers
FROM segment_trends_daily_mv
GROUP BY Segment, `Peak Season Indicator`, Season
ORDER BY Segment, `Peak Season Indicator`, MEASURE(`Total Revenue`) DESC;


Segment,Peak Season Indicator,Season,revenue,avg_order_value,customers
loyal,Peak Season,Winter,9564712.809999995,281.4061258054075,7959
loyal,Peak Season,Fall,4481380.569999996,279.98129264025965,4588
loyal,Regular Season,Winter,3537691.9199999887,281.5288811077502,3795
new,Peak Season,Winter,2937619.5300000017,272.1277934228811,5206
new,Peak Season,Fall,1461176.0400000007,271.4426973806429,3023
new,Regular Season,Winter,1019089.9799999997,269.5292197831261,2195
premium,Peak Season,Winter,5689746.089999983,284.85761940522593,4612
premium,Peak Season,Fall,2980110.7300000046,281.7805153177009,2782
premium,Regular Season,Winter,2119444.5700000003,283.8415119860721,2142
regular,Peak Season,Winter,7224968.269999987,276.3845403771848,9846
